In [1]:
# load libraries

import pandas as pd
import numpy as np
import re
import datetime
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
import xgboost as xgb

pd.set_option('display.max_colwidth',100)

/home/nidhi/miniconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#load data
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [5]:
from IPython.core.display import HTML
HTML("""
<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style>
""")



In [6]:
train.shape

(108129, 14)

In [7]:
test.shape

(63465, 12)

In [8]:
train.head()

,project_id,name,desc,goal,keywords,disable_communication,country,currency,deadline,state_changed_at,created_at,launched_at,backers_count,final_status
0,kkst1451568084,drawing for dollars,I like drawing pictures. and then i color them too. so i thought i would suggest something for m...,20.0,drawing-for-dollars,False,US,USD,1241333999,1241334017,1240600507,1240602723,3,1
1,kkst1474482071,Sponsor Dereck Blackburn (Lostwars) Artist in Residence in Kankakee Illinois,"I, Dereck Blackburn will be taking upon an incredible journey in the month of May 2009. I will b...",300.0,sponsor-dereck-blackburn-lostwars-artist-in-residence-in-kankakee-illinois,False,US,USD,1242429000,1242432018,1240960224,1240975592,2,0
2,kkst183622197,Mr. Squiggles,"So I saw darkpony's successfully funded drawing for dollars project and I thought """"""""""""""""""""""""""""...",30.0,mr-squiggles,False,US,USD,1243027560,1243027818,1242163613,1242164398,0,0
3,kkst597742710,Help me write my second novel.,Do your part to help out starving artists and help me write my second novel!\r\n\r\nI have just ...,500.0,help-me-write-my-second-novel,False,US,USD,1243555740,1243556121,1240963795,1240966730,18,1
4,kkst1913131122,Support casting my sculpture in bronze,"I'm nearing completion on a sculpture, currently titled """"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""...",2000.0,support-casting-my-sculpture-in-bronze,False,US,USD,1243769880,1243770317,1241177914,1241180541,1,0


In [9]:
train.describe()

,goal,deadline,state_changed_at,created_at,launched_at,backers_count,final_status
count,1.081290e+05,1.081290e+05,1.081290e+05,1.081290e+05,1.081290e+05,108129.000000,108129.000000
mean,3.672623e+04,1.380248e+09,1.380153e+09,1.374037e+09,1.377299e+09,123.516661,0.319627
std,9.719027e+05,4.270222e+07,4.266402e+07,4.272310e+07,4.294421e+07,1176.745162,0.466334
min,1.000000e-02,1.241334e+09,1.241334e+09,1.240335e+09,1.240603e+09,0.000000,0.000000
25%,2.000000e+03,1.346732e+09,1.346695e+09,1.340058e+09,1.343917e+09,2.000000,0.000000
50%,5.000000e+03,1.393628e+09,1.393567e+09,1.384445e+09,1.390870e+09,17.000000,0.000000
75%,1.300000e+04,1.415719e+09,1.415548e+09,1.409623e+09,1.412807e+09,65.000000,1.000000
max,1.000000e+08,1.433097e+09,1.433097e+09,1.432325e+09,1.432658e+09,219382.000000,1.000000


In [10]:
unix_cols = ['deadline','state_changed_at','launched_at','created_at']

for x in unix_cols:
    train[x] = train[x].apply(lambda k: datetime.datetime.fromtimestamp(int(k)).strftime('%Y-%m-%d %H:%M:%S'))
    test[x] = test[x].apply(lambda k: datetime.datetime.fromtimestamp(int(k)).strftime('%Y-%m-%d %H:%M:%S'))


In [11]:
train.head()

,project_id,name,desc,goal,keywords,disable_communication,country,currency,deadline,state_changed_at,created_at,launched_at,backers_count,final_status
0,kkst1451568084,drawing for dollars,I like drawing pictures. and then i color them too. so i thought i would suggest something for m...,20.0,drawing-for-dollars,False,US,USD,2009-05-03 12:29:59,2009-05-03 12:30:17,2009-04-25 00:45:07,2009-04-25 01:22:03,3,1
1,kkst1474482071,Sponsor Dereck Blackburn (Lostwars) Artist in Residence in Kankakee Illinois,"I, Dereck Blackburn will be taking upon an incredible journey in the month of May 2009. I will b...",300.0,sponsor-dereck-blackburn-lostwars-artist-in-residence-in-kankakee-illinois,False,US,USD,2009-05-16 04:40:00,2009-05-16 05:30:18,2009-04-29 04:40:24,2009-04-29 08:56:32,2,0
2,kkst183622197,Mr. Squiggles,"So I saw darkpony's successfully funded drawing for dollars project and I thought """"""""""""""""""""""""""""...",30.0,mr-squiggles,False,US,USD,2009-05-23 02:56:00,2009-05-23 03:00:18,2009-05-13 02:56:53,2009-05-13 03:09:58,0,0
3,kkst597742710,Help me write my second novel.,Do your part to help out starving artists and help me write my second novel!\r\n\r\nI have just ...,500.0,help-me-write-my-second-novel,False,US,USD,2009-05-29 05:39:00,2009-05-29 05:45:21,2009-04-29 05:39:55,2009-04-29 06:28:50,18,1
4,kkst1913131122,Support casting my sculpture in bronze,"I'm nearing completion on a sculpture, currently titled """"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""...",2000.0,support-casting-my-sculpture-in-bronze,False,US,USD,2009-05-31 17:08:00,2009-05-31 17:15:17,2009-05-01 17:08:34,2009-05-01 17:52:21,1,0


In [12]:
cols_to_use = ['name','desc']
len_feats = ['name_len','desc_len']
count_feats = ['name_count','desc_count']

for i in np.arange(2):
    train[len_feats[i]] = train[cols_to_use[i]].apply(str).apply(len)
    test[len_feats[i]] = test[cols_to_use[i]].apply(str).apply(len)


In [14]:
train.head()

,project_id,name,desc,goal,keywords,disable_communication,country,currency,deadline,state_changed_at,created_at,launched_at,backers_count,final_status,name_len,desc_len
0,kkst1451568084,drawing for dollars,I like drawing pictures. and then i color them too. so i thought i would suggest something for m...,20.0,drawing-for-dollars,False,US,USD,2009-05-03 12:29:59,2009-05-03 12:30:17,2009-04-25 00:45:07,2009-04-25 01:22:03,3,1,19,134
1,kkst1474482071,Sponsor Dereck Blackburn (Lostwars) Artist in Residence in Kankakee Illinois,"I, Dereck Blackburn will be taking upon an incredible journey in the month of May 2009. I will b...",300.0,sponsor-dereck-blackburn-lostwars-artist-in-residence-in-kankakee-illinois,False,US,USD,2009-05-16 04:40:00,2009-05-16 05:30:18,2009-04-29 04:40:24,2009-04-29 08:56:32,2,0,76,137
2,kkst183622197,Mr. Squiggles,"So I saw darkpony's successfully funded drawing for dollars project and I thought """"""""""""""""""""""""""""...",30.0,mr-squiggles,False,US,USD,2009-05-23 02:56:00,2009-05-23 03:00:18,2009-05-13 02:56:53,2009-05-13 03:09:58,0,0,13,385
3,kkst597742710,Help me write my second novel.,Do your part to help out starving artists and help me write my second novel!\r\n\r\nI have just ...,500.0,help-me-write-my-second-novel,False,US,USD,2009-05-29 05:39:00,2009-05-29 05:45:21,2009-04-29 05:39:55,2009-04-29 06:28:50,18,1,30,131
4,kkst1913131122,Support casting my sculpture in bronze,"I'm nearing completion on a sculpture, currently titled """"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""...",2000.0,support-casting-my-sculpture-in-bronze,False,US,USD,2009-05-31 17:08:00,2009-05-31 17:15:17,2009-05-01 17:08:34,2009-05-01 17:52:21,1,0,38,384


In [15]:
train['name_count'] = train['name'].str.split().str.len()
train['desc_count'] = train['desc'].str.split().str.len()

test['name_count'] = test['name'].str.split().str.len()
test['desc_count'] = test['desc'].str.split().str.len()

In [16]:
train['keywords_len'] = train['keywords'].str.len()
train['keywords_count'] = train['keywords'].str.split('-').str.len()

test['keywords_len'] = test['keywords'].str.len()
test['keywords_count'] = test['keywords'].str.split('-').str.len()

In [17]:
train.head()

,project_id,name,desc,goal,keywords,disable_communication,country,currency,deadline,state_changed_at,created_at,launched_at,backers_count,final_status,name_len,desc_len,name_count,desc_count,keywords_len,keywords_count
0,kkst1451568084,drawing for dollars,I like drawing pictures. and then i color them too. so i thought i would suggest something for m...,20.0,drawing-for-dollars,False,US,USD,2009-05-03 12:29:59,2009-05-03 12:30:17,2009-04-25 00:45:07,2009-04-25 01:22:03,3,1,19,134,3.0,26.0,19,3
1,kkst1474482071,Sponsor Dereck Blackburn (Lostwars) Artist in Residence in Kankakee Illinois,"I, Dereck Blackburn will be taking upon an incredible journey in the month of May 2009. I will b...",300.0,sponsor-dereck-blackburn-lostwars-artist-in-residence-in-kankakee-illinois,False,US,USD,2009-05-16 04:40:00,2009-05-16 05:30:18,2009-04-29 04:40:24,2009-04-29 08:56:32,2,0,76,137,10.0,25.0,74,10
2,kkst183622197,Mr. Squiggles,"So I saw darkpony's successfully funded drawing for dollars project and I thought """"""""""""""""""""""""""""...",30.0,mr-squiggles,False,US,USD,2009-05-23 02:56:00,2009-05-23 03:00:18,2009-05-13 02:56:53,2009-05-13 03:09:58,0,0,13,385,2.0,22.0,12,2
3,kkst597742710,Help me write my second novel.,Do your part to help out starving artists and help me write my second novel!\r\n\r\nI have just ...,500.0,help-me-write-my-second-novel,False,US,USD,2009-05-29 05:39:00,2009-05-29 05:45:21,2009-04-29 05:39:55,2009-04-29 06:28:50,18,1,30,131,6.0,24.0,29,6
4,kkst1913131122,Support casting my sculpture in bronze,"I'm nearing completion on a sculpture, currently titled """"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""...",2000.0,support-casting-my-sculpture-in-bronze,False,US,USD,2009-05-31 17:08:00,2009-05-31 17:15:17,2009-05-01 17:08:34,2009-05-01 17:52:21,1,0,38,384,6.0,18.0,38,6


In [18]:
unix_cols = ['deadline','state_changed_at','launched_at','created_at']

for x in unix_cols:
    train[x] = train[x].apply(lambda k: datetime.datetime.strptime(k, '%Y-%m-%d %H:%M:%S'))
    test[x] = test[x].apply(lambda k: datetime.datetime.strptime(k, '%Y-%m-%d %H:%M:%S'))


In [38]:
final = train['final_status'].value_counts()
communic=train['disable_communication'].value_counts()
# final_a = train[train['final_status']==0]['disable_communication'].value_counts()
# df = pd.DataFrame([final, final_a])
# df.index = ['Survived','Dead']
# df.plot(kind='bar',stacked=True)
print final, "\n"
print communic



0    73568
1    34561
Name: final_status, dtype: int64 

False    107806
True        323
Name: disable_communication, dtype: int64


In [41]:
df=pd.DataFrame(final, communic)
print df

                       final_status
disable_communication              
107806                          NaN
323                             NaN


In [42]:
unix_cols = ['deadline','state_changed_at','launched_at','created_at']

for x in unix_cols:
    train[x] = train[x].apply(lambda k: datetime.datetime.strptime(k, '%Y-%m-%d %H:%M:%S'))
    test[x] = test[x].apply(lambda k: datetime.datetime.strptime(k, '%Y-%m-%d %H:%M:%S'))


TypeError: strptime() argument 1 must be string, not Timestamp

In [43]:

time1 = []
time3 = []
for i in np.arange(train.shape[0]):
    time1.append(np.round((train.loc[i, 'launched_at'] - train.loc[i, 'created_at']).total_seconds()).astype(int))
    time3.append(np.round((train.loc[i, 'deadline'] - train.loc[i, 'launched_at']).total_seconds()).astype(int))


In [44]:
train['time1'] = np.log(time1)
train['time3'] = np.log(time3)

In [45]:
# for test data
time5 = []
time6 = []
for i in np.arange(test.shape[0]):
    time5.append(np.round((test.loc[i, 'launched_at'] - test.loc[i, 'created_at']).total_seconds()).astype(int))
    time6.append(np.round((test.loc[i, 'deadline'] - test.loc[i, 'launched_at']).total_seconds()).astype(int))

In [46]:
test['time1'] = np.log(time5)
test['time3'] = np.log(time6)

In [47]:
feat = ['disable_communication','country']

for x in feat:
    le = LabelEncoder()
    le.fit(list(train[x].values) + list(test[x].values))
    train[x] = le.transform(list(train[x]))
    test[x] = le.transform(list(test[x]))

In [48]:
train['goal'] = np.log1p(train['goal'])
test['goal'] = np.log1p(test['goal'])


In [49]:
kickdesc = pd.Series(train['desc'].tolist() + test['desc'].tolist()).astype(str)

In [50]:
# this function cleans punctuations, digits and irregular tabs. Then converts the sentences to lower
def desc_clean(word):
    p1 = re.sub(pattern='(\W+)|(\d+)|(\s+)',repl=' ',string=word)
    p1 = p1.lower()
    return p1

kickdesc = kickdesc.map(desc_clean)

In [51]:
stop = set(stopwords.words('english'))
kickdesc = [[x for x in x.split() if x not in stop] for x in kickdesc]

stemmer = SnowballStemmer(language='english')
kickdesc = [[stemmer.stem(x) for x in x] for x in kickdesc]

kickdesc = [[x for x in x if len(x) > 2] for x in kickdesc]

kickdesc = [' '.join(x) for x in kickdesc]

In [63]:
cv = CountVectorizer(max_features=500)

In [64]:
alldesc = cv.fit_transform(kickdesc).todense()


In [65]:
#create a data frame
combine = pd.DataFrame(alldesc)
combine.rename(columns= lambda x: 'variable_'+ str(x), inplace=True)


In [66]:
train_text = combine[:train.shape[0]]
test_text = combine[train.shape[0]:]

test_text.reset_index(drop=True,inplace=True)


In [69]:
cols_to_use = ['name_len','desc_len','keywords_len','name_count','desc_count','keywords_count','time1','time3','goal']

In [70]:
train = train.loc[:,cols_to_use]
test = test.loc[:,cols_to_use]


In [71]:
X_train = pd.concat([train, train_text],axis=1)
X_test = pd.concat([test, test_text],axis=1)


In [72]:
print X_train.shape
print X_test.shape

(108129, 509)
(63465, 509)


In [73]:
dtrain = xgb.DMatrix(data=X_train, label = target)
dtest = xgb.DMatrix(data=X_test)

In [74]:
params = {
    'objective':'binary:logistic',
    'eval_metric':'error',
    'eta':0.025,
    'max_depth':6,
    'subsample':0.7,
    'colsample_bytree':0.7,
    'min_child_weight':5
    
}

In [75]:
# You can probably get better accuracy with rounds > 1000. 
bst = xgb.cv(params, dtrain, num_boost_round=1000, early_stopping_rounds=40,nfold=5L,verbose_eval=10)



[0]	train-error:0.311533+0.000372653	test-error:0.313887+0.00276366
[10]	train-error:0.309753+0.00139237	test-error:0.313646+0.00311179
[20]	train-error:0.308381+0.000856488	test-error:0.312814+0.00352685
[30]	train-error:0.307251+0.00130237	test-error:0.311861+0.00357432
[40]	train-error:0.305713+0.00100673	test-error:0.310853+0.00295552
[50]	train-error:0.304261+0.0011377	test-error:0.309521+0.00261851
[60]	train-error:0.302837+0.000676381	test-error:0.308356+0.00258093
[70]	train-error:0.301158+0.000426749	test-error:0.307311+0.00239366
[80]	train-error:0.299843+0.000527155	test-error:0.3067+0.00211246
[90]	train-error:0.298814+0.000449077	test-error:0.306007+0.00178264
[100]	train-error:0.297834+0.000409762	test-error:0.305036+0.00205358
[110]	train-error:0.297045+0.000558994	test-error:0.304129+0.00197847
[120]	train-error:0.296046+0.000545838	test-error:0.303168+0.00191676
[130]	train-error:0.295013+0.000579647	test-error:0.302354+0.00202776
[140]	train-error:0.294312+0.000530088

In [76]:
bst_train = xgb.train(params, dtrain, num_boost_round=1000)


In [77]:
p_test = bst_train.predict(dtest)


In [79]:
sub = pd.DataFrame()
test.head()

,name_len,desc_len,keywords_len,name_count,desc_count,keywords_count,time1,time3,goal
0,10,119,8,1,19.0,1,11.964027,15.056664,8.853808
1,16,119,16,2,20.0,2,11.646556,14.922091,10.463132
2,43,126,43,7,19.0,7,11.416106,14.949674,10.809748
3,58,108,49,8,17.0,6,14.195786,14.767940,10.596660
4,18,132,18,3,20.0,3,14.759266,15.132977,9.903538


In [80]:
p_test

array([ 0.2177182 ,  0.0997297 ,  0.11680372, ...,  0.15612027,
        0.08667103,  0.38291654], dtype=float32)

In [81]:
sub['final_status'] = p_test

In [83]:
sub.to_csv("xgb_with_python_feats.csv",index=False)